In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# --- 1. Load the preprocessed data ---
try:
    data = pd.read_csv('final_data_for_modeling.csv')
    print("Successfully loaded 'final_data_for_modeling.csv'.")
    print("Dataset shape:", data.shape)
except FileNotFoundError:
    print("Error: 'final_data_for_modeling.csv' not found. Please run the data preprocessing script first.")
    exit()

# --- 2. Define features (X) and target (Y) ---
target_column = 'PM2.5'

# The features will be all columns except for 'Country', 'Year', and the target variable.
feature_columns = data.drop(columns=['Country', 'Year', target_column]).columns
X = data[feature_columns]
y = data[target_column]
print("\nShape of feature variables (X):", X.shape)
print("Shape of target variable (y):", y.shape)


# --- 3. Split the dataset (80/20 split) ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nTraining set size: {X_train.shape[0]} records")
print(f"Test set size: {X_test.shape[0]} records")


# --- 4. Feature Scaling (StandardScaler) ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("\nFeature data has been standardized.")


# --- 5. Model Training and Evaluation ---
results = {}

# Model 1: Linear Regression (Baseline Model)
print("\n--- Training Linear Regression Model ---")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
lr_predictions = lr_model.predict(X_test_scaled)
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_predictions))
lr_r2 = r2_score(y_test, lr_predictions)
results['Linear Regression'] = {'RMSE': lr_rmse, 'R2': lr_r2}
print(f"Linear Regression - RMSE: {lr_rmse:.4f}, R²: {lr_r2:.4f}")

# Model 2: Random Forest Regressor (Default Parameters)
print("\n--- Training Random Forest Model (Default Parameters) ---")
rf_model_default = RandomForestRegressor(random_state=42, n_jobs=-1)
rf_model_default.fit(X_train_scaled, y_train)
rf_default_predictions = rf_model_default.predict(X_test_scaled)
rf_default_rmse = np.sqrt(mean_squared_error(y_test, rf_default_predictions))
rf_default_r2 = r2_score(y_test, rf_default_predictions)
results['Random Forest (Default)'] = {'RMSE': rf_default_rmse, 'R2': rf_default_r2}
print(f"Random Forest (Default) - RMSE: {rf_default_rmse:.4f}, R²: {rf_default_r2:.4f}")


# --- 6. Hyperparameter Tuning (GridSearchCV) ---
print("\n--- Optimizing Random Forest with GridSearchCV ---")
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5]
}
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
                           param_grid=param_grid,
                           cv=3,
                           scoring='r2',
                           verbose=1) # Set verbose to 1 for cleaner output
grid_search.fit(X_train_scaled, y_train)

print(f"\nBest parameters found: {grid_search.best_params_}")
best_rf_model = grid_search.best_estimator_
best_rf_predictions = best_rf_model.predict(X_test_scaled)
best_rf_rmse = np.sqrt(mean_squared_error(y_test, best_rf_predictions))
best_rf_r2 = r2_score(y_test, best_rf_predictions)
results['Random Forest (Optimized)'] = {'RMSE': best_rf_rmse, 'R2': best_rf_r2}
print(f"Random Forest (Optimized) - RMSE: {best_rf_rmse:.4f}, R²: {best_rf_r2:.4f}")


# --- 7. Summarize and Display Results ---
print("\n--- Model Performance Summary (for report table) ---")
results_df = pd.DataFrame(results).T
print(results_df)


# --- 8. Feature Importance Analysis and Visualization ---
print("\n--- Generating Feature Importance Chart ---")
importances = best_rf_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df.head(10), palette='viridis')
plt.title('Top 10 Most Important Features for Predicting PM2.5 (Optimized Random Forest)')
plt.xlabel('Importance Score')
plt.ylabel('Industrial Sector')
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300)
print("Feature importance chart saved as 'feature_importance.png'")
plt.show()